In [ ]:
#| default_exp interactive_app

# AnDi Challenge 2020 - Interactive tool

Explore the results of the AnDi challenge 2020 interactively. Here you will find access to the details of each submissions results over the different features of the dataset. 
This tab showcases a summary of the results of Task 1 and Task 2. The rest of the tabs corresponds to each of the tasks separatedely.

# Packages and globals

In [ ]:
#| hide 
from bokeh.plotting import figure
from bokeh.io import curdoc, show, output_notebook
from bokeh.layouts import row,gridplot,grid, column
from bokeh.models import ColumnDataSource, RadioButtonGroup, Select, Text, Slider, RangeSlider, PreText, FileInput, LinearColorMapper, HoverTool, Label, Span, Div
from bokeh.events import Tap
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.palettes import Category10 as palette

import numpy as np
import pandas as pd

from bokeh.tile_providers import OSM, get_provider
from bokeh.models.widgets import Tabs, Panel

from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc
from sklearn import preprocessing

from scipy.stats import gaussian_kde

import io

from pybase64 import b64decode

import numpy as np

from functools import partial 

In [ ]:
#| hide
processes = ['attm', 'ctrw', 'fbm', 'lw', 'sbm']
processes_simp = ['a','c','f','l','s']
dims = ["1D", "2D", "3D"]
colors_ROC = palette[7]

In [ ]:
#| hide
# Checks if we are in notebook or .py and executes correct dfs
try:
    __IPYTHON__
    
    df_t1 = [pd.read_csv(f'csv/task1_{dim}d.csv') for dim in [1,2,3]]
    df_t2 = [pd.read_csv(f'csv/task2_{dim}d.csv') for dim in [1,2,3]]
    df_t3 = [pd.read_csv(f'csv/task3_{dim}d.csv') for dim in [1,2,3]]

    df_t1_results = [pd.read_csv(f'csv/results_parameters_task1_{dim}D.csv') for dim in [1,2,3]]
    df_t2_results = [pd.read_csv(f'csv/results_parameters_task2_{dim}D.csv') for dim in [1,2,3]]
    
    notebook = True

except NameError:
    
    df_t1 = [pd.read_csv(f'source_nbs/interactive_app/csv/task1_{dim}d.csv') for dim in [1,2,3]]
    df_t2 = [pd.read_csv(f'source_nbs/interactive_app/csv/task2_{dim}d.csv') for dim in [1,2,3]]
    df_t3 = [pd.read_csv(f'source_nbs/interactive_app/csv/task3_{dim}d.csv') for dim in [1,2,3]]

    df_t1_results = [pd.read_csv(f'source_nbs/interactive_app/csv/results_parameters_task1_{dim}D.csv') for dim in [1,2,3]]
    df_t2_results = [pd.read_csv(f'source_nbs/interactive_app/csv/results_parameters_task2_{dim}D.csv') for dim in [1,2,3]]
    
    notebook = False

In [ ]:
  
#| hide 
def updated_preds(df, team, model = None, a_min = None, a_max = None, l_min = 0, l_max = 1000, loc = 'All'):
    # Exponent
    if a_min != None:
        df = df.loc[lambda df: (df['alpha'] >= a_min) & (df['alpha'] <= a_max)]
    # Length
    df = df.loc[lambda df: (df['length'] >= l_min) & (df['length'] <= l_max)]
    # Model
    if model != 'All' and model != None:
        model = processes.index(model)
        df = df.loc[lambda df: (df['model'] == model)]
    # Localization
    if loc != 'All':
        loc = float(loc)
        df = df.loc[lambda df: (df['loc_error'] == loc)]    
    return df[team]

# Task 1

In [ ]:
  
#| hide 
def update_t1():
    # Data
    preds = updated_preds(df = df_t1[dims.index(dim_select_t1.value)], 
              model = model_group.value, 
              a_min = alpha_slider_t1.value[0],
              a_max = alpha_slider_t1.value[1],
              l_min = length_slider_t1.value[0],
              l_max = length_slider_t1.value[1],
              loc = noise_group_t1.value,
              team = team_select_t1.value)      
    
    if preds.shape[0] == 0:
         # If there are no enough trajectories
        div_t1_aux.text = f"""
        <p style="color:red">  <b>Warning!</b> <br> Not enough trajectories. Select wider option ranges.</p>"""
        return
    
    trues = df_t1[dims.index(dim_select_t1.value)]['alpha'][preds.index]
    # hist 2d
    fig_exp.renderers = []
    r, bins = fig_exp.hexbin(trues, 
                             preds, 
                             size=0.05, hover_color="pink", hover_alpha=0.8, palette='Viridis256')
    # kde 
    fig_kde.renderers = []
    data = (preds-trues).values
    density = gaussian_kde(data)
    hist, edges = np.histogram(data, density=True, bins=50)
    fig_kde.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color = '#440154', line_color="white", alpha = 0.5)
    xs = np.linspace(edges[0],edges[-1],100)
    fig_kde.line(xs, density(xs), line_color = '#79D151', line_width = 3)
    
    #MAE vs MOD
    fig_mod.renderers = []
    mae_vs_mod(fig_mod)
    
    # Text
    div_t1_aux.text = f"""
    <b>Prediction summary </b>     
    <br>
    Number trajs. = {len(trues)}
    <br>
    MAE = {round((preds-trues).abs().mean(),3)}"""
    

In [ ]:
 
#| hide
def mae_vs_mod(fig):    
    kwargs = {'df' : df_t1[dims.index(dim_select_t1.value)],
              'a_min' : alpha_slider_t1.value[0], 'a_max' : alpha_slider_t1.value[1],
              'l_min' : length_slider_t1.value[0],'l_max' : length_slider_t1.value[1],
              'loc' : noise_group_t1.value,
              'team' : team_select_t1.value}
    mae_mod = []
    if model_group.value != 'All':
        pr = [model_group.value]
    else:
        if alpha_slider_t1.value[0] > 1:
            pr = np.delete(processes, [0,1])
        elif alpha_slider_t1.value[1] < 1:
            pr = np.delete(processes, 3)
        elif alpha_slider_t1.value[0] == 2:
            pr = np.delete(processes, [0,1,2])
        else:
            pr = processes
        
    for idx, mod in enumerate(pr):
        preds = updated_preds(model = mod, **kwargs)
        trues = df_t1[dims.index(dim_select_t1.value)]['alpha'][preds.index]
        mae_mod.append(np.abs((preds-trues).values).mean())
        
    fig.circle(pr, mae_mod, size = 10, color = '#440154')
        

### Figure

In [ ]:
#| echo: false

# init bokeh
output_notebook()

# Widgets
dim_select_t1 = Select(value="1D", options=dims, title = 'Dimensions')
team_select_t1 = Select(value=df_t1[0].keys()[6:].tolist()[0], options=df_t1[0].keys()[6:].tolist(), title = 'Team')
alpha_slider_t1 = RangeSlider(start=0, end=2, value=(0, 2), step=.05, title=r"Anomalous exponent")
length_slider_t1 = RangeSlider(start=10, end=1000, value=(10, 1000), step=1, title=r"Length")
model_group = Select(options=['All']+[p for p in processes], value="All", title ='Diffusion model')
noise_group_t1 = Select(options=['All']+[str(n) for n in np.unique(df_t1[0].loc_error)[::-1]], value="All", title ='Signal to Noise ratio (SNR)')

# Text
div_t1 = Div(text= """ <h2>Task 1 - Inference of the anomalous diffusion exponent</h2>

The goal of this task is to extract the anomalous diffusion exponent, usually defined as the scaling exponent of the ensemble averaged mean squared displacement w.r.t to time. 
The central figure shows a 2D histogram comparing the predicted and ground-truth exponents. 
The upper-rightmost figure shows the histogram over the difference between these two. 
The lower-rightmost figure shows the MAE of the method for each diffusion model.
""", width = 1000, margin = (0,0,20,0))

div_t1_aux = Div(text= "asd", width = 220)


# Plots
size_t1 = 500
fig_exp = figure(tools="pan,wheel_zoom,reset", 
                 match_aspect=True, 
                 background_fill_color='#440154', 
                 x_range = [0,2.05], y_range = [0,2.05],
                 x_axis_label = 'True exponent', y_axis_label = 'Predicted exponent',
                 plot_height = size_t1, plot_width = size_t1, margin=(0, 20, 0, 20))
fig_exp.grid.visible = False
# fig_exp.background_fill_color=None

fig_kde = figure(y_axis_label = 'Frequency',
                 x_axis_label = r'Difference between predicted and true exponents',
                 plot_height = int(size_t1/1.7),
                 x_range = [-1, 1],
                 plot_width = size_t1,
                 margin = (0,0,20,0))
fig_kde.background_fill_color=None

fig_mod = figure(y_axis_label = 'MAE',
                 x_axis_label = r'Diffusion model', 
                 x_range=processes, 
                 plot_height = int(size_t1/2.7), 
                 plot_width = size_t1 )
fig_mod.background_fill_color=None

# Updates
update_t1()

def update_select_team_t1(attrname, old, new):
    team_select_t1.options = df_t1[dims.index(dim_select_t1.value)].keys()[6:].tolist()  
    team_select_t1.value = team_select_t1.options[0]    
    
    noise_group_t1.options = ['All']+[str(n) for n in np.unique(df_t1[dims.index(dim_select_t1.value)].loc_error)[::-1]]
    noise_group_t1.value = 'All'
    
    update_t1()

controls_t1 = [dim_select_t1, team_select_t1, model_group, noise_group_t1]
for control in controls_t1:
    control.on_change('value', lambda attr, old, new: update_t1() )
    
sliders_t1 = [alpha_slider_t1, length_slider_t1]
for control in sliders_t1:
    control.on_change('value_throttled', lambda attr, old, new: update_t1())
all_controls_t1 = controls_t1+sliders_t1

dim_select_t1.on_change('value', update_select_team_t1)

# Layout
inputs_t1 = column(*all_controls_t1+[div_t1_aux], width = 220, height = 500)

l1 = grid([div_t1,
           row(inputs_t1, fig_exp, column(fig_kde, fig_mod))])

def modify_doc(doc):
    doc.add_root(row(l1, width=10))

handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)

Loading BokehJS ...

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=4321&bokeh-absolute-url=http://localhost:58478&resources=none (::1)
HTTPServerRequest(protocol='http', host='localhost:58478', method='GET', uri='/autoload.js?bokeh-autoload-element=4321&bokeh-absolute-url=http://localhost:58478&resources=none', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "C:\Users\Gorka\anaconda3\lib\site-packages\tornado\web.py", line 1704, in _execute
    result = await result
  File "C:\Users\Gorka\anaconda3\lib\site-packages\bokeh\server\views\autoload_js_handler.py", line 62, in get
    session = await self.get_session()
  File "C:\Users\Gorka\anaconda3\lib\site-packages\bokeh\server\views\session_handler.py", line 144, in get_session
    session = await self.application_context.create_session_if_needed(session_id, self.request, token)
  File "C:\Users\Gorka\anaconda3\lib\site-packages\bokeh\server\contexts.py", line 243, in create_session_if_ne

# Task 2

In [ ]:
 
#| hide
def conf_mat(true, pred, p):
    conf = confusion_matrix(true, pred, normalize = 'true', labels = [0,1,2,3,4])
    tn, pn, alpha, color, posx, posy, text = [], [], [], [], [], [], []

    for idxt, true in enumerate(processes):
        for idxp, pred in enumerate(processes):
            tn.append(true)
            pn.append(pred)
            alpha.append(conf[idxt, idxp])
            color.append('#FF5733')
            posx.append(idxt+0.4)
            posy.append(4-idxp+0.4)
            text.append(str(round(conf[idxt, idxp],2)))


    source = ColumnDataSource(
        data=dict(predicted=pn, actual=tn, count=conf.flatten(), colors = color, alphas = alpha, posx = posx, posy = posy, text = text)
    )
    p.plot_width = 600
    p.plot_height = p.plot_width
    rectwidth = 0.9
    p.rect('actual','predicted', rectwidth, rectwidth, source=source,
          color='colors', alpha='alphas',line_width=1)
    glyph = Text(x="posx", y="posy", text="text", text_color = '#000000')
    p.add_glyph(source, glyph)
    

In [ ]:
 
#| hide
def ROC(fig):       

    preds = updated_preds_t2_probs(df = df_t2[dims.index(dim_select_t2.value)],
                                  a_min = alpha_slider_t2.value[0],
                                  a_max = alpha_slider_t2.value[1],
                                  l_min = length_slider_t2.value[0],
                                  l_max = length_slider_t2.value[1],
                                  loc = noise_group_t2.value,
                                  team = team_select_t2.value)       
    trues = df_t2[dims.index(dim_select_t2.value)]['model'][preds.index] 

    lb = preprocessing.LabelBinarizer()
    lb.fit(trues.values)

    y_test = lb.transform(trues.values)
    y_score = preds.to_numpy()[:,np.unique(trues.values).astype(int)]

    classes = np.unique(trues.values).astype(int)
    n_classes = len(classes)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    source_macro = ColumnDataSource(data=dict(fpr_mac = fpr["macro"], tpr_mac = tpr["macro"]))
    source_micro = ColumnDataSource(data=dict(fpr_mic = fpr["micro"], tpr_mic = tpr["micro"]))

    
    for i, clas in enumerate(classes):
        fig.line(fpr[i], tpr[i], color = colors_ROC[clas], legend_label = processes[clas], line_width = 2)
    fig.line(x="fpr_mac", y="tpr_mac", source=source_macro, color = 'black', line_width = 2, legend_label = 'Macro')
    fig.line(x="fpr_mic", y="tpr_mic", source=source_micro, color = 'black', line_dash = 'dashed', line_width = 2, legend_label = 'Micro')
    fig.legend.location = "bottom_right"
    
    return roc_auc['micro']



In [ ]:
 
#| hide
def updated_preds_t2_probs(df, team, a_min = None, a_max = None, l_min = 0, l_max = 1000, loc = 'All'):
    # Exponent
    if a_min != None:
        df = df.loc[lambda df: (df['alpha'] >= a_min) & (df['alpha'] <= a_max)]
    # Length
    df = df.loc[lambda df: (df['length'] >= l_min) & (df['length'] <= l_max)]
    # Localization
    if loc != 'All':
        loc = float(loc)
        df = df.loc[lambda df: (df['loc_error'] == loc)]  
        
    filter_col = [col for col in df if col.startswith(team+'_')]
    
    return df[filter_col]

In [ ]:
  
#| hide
def update_t2():
    # Data
    preds = updated_preds(df = df_t2[dims.index(dim_select_t2.value)], 
          a_min = alpha_slider_t2.value[0],
          a_max = alpha_slider_t2.value[1],
          l_min = length_slider_t2.value[0],
          l_max = length_slider_t2.value[1],
          loc = noise_group_t2.value,
          team = team_select_t2.value)
    
    if preds.shape[0] == 0:
         # If there are no enough trajectories
        div_t2_aux.text = f"""
        <p style="color:red">  <b>Warning!</b> <br> Not enough trajectories. Select wider option ranges.</p>"""
        return

    trues = df_t2[dims.index(dim_select_t2.value)]['model'][preds.index] 
    # Confusion matrix
    fig_conf.renderers = []
    conf_mat(trues, preds, fig_conf)  
    # ROC
    fig_roc.renderers = []   
    fig_roc.legend.items = []
    auc = ROC(fig_roc)
    # Text
    div_t2_aux.text = f""" <b>Prediction summary </b>     
    <br>
    Number trajs. = {len(trues)} 
    <br>
    F1-score = {round(f1_score(trues, preds, average = 'micro'),3)}
    <br>
    AUC = {round(auc,2)}"""

### Figure

In [ ]:
#| hide
#| exec: false
# Widgets
dim_select_t2 = Select(value="1D", options=dims, title = 'Dimension')
teams_t2 = [s.split('_')[0] for s in df_t2[0].keys()[6:].tolist()][0:-1:6]
team_select_t2 = Select(value=teams_t2[0], options=teams_t2, title = 'Team')
alpha_slider_t2 = RangeSlider(start=0, end=2, value=(0, 2), step=.05, title=r"Anomalous exponent")
length_slider_t2 = RangeSlider(start=10, end=1000, value=(10, 1000), step=1, title=r"Length")
noise_group_t2 = Select(options=['All']+[str(n) for n in np.unique(df_t2[0].loc_error)[::-1]], value="All", title ='Signal to Noise ratio (SNR)')


# Text
div_t2 = Div(text= f""" <h2>Task 2 - Diffusion model classification</h2>

The goal of this task is to classify the trajectories among five diffusion models.
The central figure shows the confusion matrix of the method. 
The rightmost figure shows the receiving operator curve (ROC) for each model and both micro and macro averaged.
""", width = 1000, margin = (0,0,20,0))

div_t2_aux = Div(text= "" , width = 220)

# Plots
size_t2 = 500
fig_conf = figure(tools="",
                  y_axis_label = 'Predicted model',
                  x_axis_label = r'True model',
                  y_range=sorted(processes, reverse=True),
                  x_range=processes,
                 plot_height = size_t2, plot_width = size_t2,
                 margin = (0,20,0,20))
fig_conf.background_fill_color=None

fig_roc = figure(x_axis_label = 'False Positive Rate', y_axis_label = 'True Positive Rate', plot_height = int(size_t2/1.5), plot_width = size_t2, margin = (0,20,0,20))   
fig_roc.background_fill_color=None

# Updates
update_t2()

def update_select_team_t2(attrname, old, new):
    teams_t2 = [s.split('_')[0] for s in df_t2[dims.index(dim_select_t2.value)].keys()[6:].tolist()][0:-1:6]
    team_select_t2.options = teams_t2  
    team_select_t2.value = team_select_t2.options[0]
    
    noise_group_t2.options = ['All']+[str(n) for n in np.unique(df_t2[dims.index(dim_select_t2.value)].loc_error)[::-1]]
    noise_group_t2.value = 'All'
    
    update_t2()    

controls_t2 = [dim_select_t2, team_select_t2, noise_group_t2]
for control in controls_t2:
    control.on_change('value', lambda attr, old, new: update_t2())
sliders_t2 = [alpha_slider_t2, length_slider_t2]
for control in sliders_t2:
    control.on_change('value_throttled', lambda attr, old, new: update_t2())
all_controls_t2 = controls_t2+sliders_t2

dim_select_t2.on_change('value', update_select_team_t2)

# Layout
inputs_t2 = column(*all_controls_t2+[div_t2_aux], width = 220, height = 500)

l2 = grid([div_t2,
           row(inputs_t2, fig_conf, fig_roc)])


if notebook:
    def modify_doc(doc):
        doc.add_root(row(l2, width=10))

    handler = FunctionHandler(modify_doc)
    app = Application(handler)
    show(app)

# New upload 

In [ ]:
 
#| hide

def upload_data(attr, old, new):   
    
    # Get data
    decoded = b64decode(new)    
    f = io.BytesIO(decoded) 
    
    correct_dim_t1, correct_dim_t2, correct_dim_t3 = [], [], []
    
    if file_input.filename[-5] == '1':        
        correct_dim_t1 = upload_t1(f)        
    if file_input.filename[-5] == '2':        
        correct_dim_t2 = upload_t2(f)
    if file_input.filename[-5] == '3':        
        correct_dim_t3 = upload_t3(f)     
    
    # Text
    div_upload_aux.text = f""" <b>You have correctly participated in: </b>     
    <br>
    T1 - Dim = {correct_dim_t1}
    <br>
    T2 - Dim = {correct_dim_t2}
    <br>
    T3 - Dim = {correct_dim_t3}"""
    
    # Update main figure
    if file_input.filename[-5] == '1' or file_input.filename[-5] == '2':
        update_main()
    


In [ ]:
 
#| hide

def upload_summary_t1(dim):

    lengths = np.arange(10,1020,50)
    alphas = np.arange(0.05, 2.1, 0.2)
    noise = [0.1, 0.5, 1]
    new_column = []
    for idxl, l in enumerate((lengths[:-1])):
        for idxa, a in enumerate(alphas[:-1]):
            for idx_m, m in enumerate(processes):
                for n in noise:  

                    preds = df_t1[dim-1][
                                    (df_t1[dim-1].model == processes.index(m)) &
                                    (df_t1[dim-1].alpha >= alphas[idxa]) &
                                    (df_t1[dim-1].alpha < alphas[idxa+1]) &
                                    (df_t1[dim-1].length >= lengths[idxl]) &
                                    (df_t1[dim-1].length < lengths[idxl+1]) &
                                    (df_t1[dim-1].loc_error == n)]
                    preds = preds['upload']
                    if len(preds) > 0:
                        trues = df_t1[dim-1]['alpha'][preds.index]
                        mae = (preds-trues).abs().mean()
                        new_column.append(mae)
                    
    df_t1_results[dim-1]['upload'] = new_column

In [ ]:
 
#| hide

def upload_summary_t2(dim):

    lengths = np.arange(10,1020,50)
    alphas = np.arange(0.05, 2.1, 0.2)
    noise = [0.1, 0.5, 1]
    new_column = []
    for idxl, l in enumerate((lengths[:-1])):
        for idxa, a in enumerate(alphas[:-1]):
                for n in noise:  

                    preds = df_t2[dim-1][
                                    (df_t2[dim-1].alpha >= alphas[idxa]) &
                                    (df_t2[dim-1].alpha < alphas[idxa+1]) &
                                    (df_t2[dim-1].length >= lengths[idxl]) &
                                    (df_t2[dim-1].length < lengths[idxl+1]) &
                                    (df_t2[dim-1].loc_error == n)]
                    preds = preds['upload']
                    if len(preds) > 0:
                        trues = df_t2[dim-1]['model'][preds.index]
                        f1 = f1_score(trues, preds, average = 'micro')
                        new_column.append(f1)
                    
    df_t2_results[dim-1]['upload'] = new_column

In [ ]:
 
#| hide

def upload_t1(f):
    
    df = pd.read_csv(f, header = None, sep = ';')    
    df.columns = ['dim', 'preds']
    
    correct_dim_t1 = []
    for dim in [1,2,3]:            
        # Read data and update main dataframe
        df_dim = df.loc[lambda df: (df['dim'] == dim)]   
        
        # If there are no prediction for this dimension
        if df_dim.shape[0] == 0: continue

        if len(df_dim) != 10000:          
            df_dim = df_dim.head(10000)  
            
        df_t1[dim-1]['upload'] = df_dim['preds'].values   
        # Saving dimensions participated
        correct_dim_t1.append(dim)
        
        # Update summary tab information
        upload_summary_t1(dim)
        
    # Add upload to controls    
    team_select_t1.options = df_t1[dims.index(dim_select_t1.value)].keys()[6:].tolist()  
    team_select_t1.value = team_select_t1.options[-1]
    
    return correct_dim_t1

In [ ]:
 
#| hide

def upload_t2(f):
    
    df = pd.read_csv(f, header = None, sep = ';') 
    df.columns = ['dim']+list(range(5))
    
    correct_dim_t2 = []
    for dim in [1,2,3]:            
        # Read data and update main dataframe
        df_dim = df.loc[lambda df: (df['dim'] == dim)]  
        
        # If there are no prediction for this dimension
        if df_dim.shape[0] == 0: continue

        if len(df_dim) != 10000:          
            df_dim = df_dim.head(10000)   

        df_t2[dim-1]['upload'] = df_dim[list(range(5))].idxmax(axis = 'columns').values

        for i in range(5):
            df_t2[dim-1]['upload_'+processes_simp[i]] = df_dim[i].values
        
        # Saving dimensions participated
        correct_dim_t2.append(dim)
        
        # Update summary tab information
        upload_summary_t2(dim)
        
        
        
    # Add upload to controls  
    teams_t2 = [s.split('_')[0] for s in df_t2[dims.index(dim_select_t2.value)].keys()[6:].tolist()][0:-1:6]
    team_select_t2.options = teams_t2  
    team_select_t2.value = team_select_t2.options[-1]
    
    return correct_dim_t2

In [ ]:
 
#| hide

def upload_t3(f):
    
    df = pd.read_csv(f, header = None, sep = ';') 
    df.columns = ['dim']+list(range(5))
    
    labels_t3 = ['cp', 'm1', 'a1', 'm2', 'a2']
    correct_dim_t3 = []
    for dim in [1,2,3]:            
        # Read data and update main dataframe
        df_dim = df.loc[lambda df: (df['dim'] == dim)]     

        # If there are no prediction for this dimension
        if df_dim.shape[0] == 0: continue

        if len(df_dim) != 10000:          
            df_dim = df_dim.head(10000)          

        for idx_l, label in enumerate(labels_t3):   
            df_t3[dim-1]['upload_'+label] = df_dim[idx_l].values

        # Saving dimensions participated
        correct_dim_t3.append(dim)
        
    # Add upload to controls
    team_select_t3.options = np.unique([n[:-3] for n in df_t3[dims.index(dim_select_t3.value)].keys()[6:]]).tolist()
    team_select_t3.value = team_select_t3.options[-1]

    return correct_dim_t3

In [ ]:
#| hide
#| exec: false
# Widgets
file_input = FileInput(accept=".txt")
file_input.on_change('value', upload_data)

# Divs
div_upload = Div(text= f""" <h2>New submission</h2>

<p align = "justify"> Score the performance of your method for the characterization of anomalous diffusion and compare it with other participants. 
<br>
<br>
Run your code to obtain predictions for the 
<a href="https://drive.google.com/drive/folders/1h2cg1RNoKbhTiiPSNCHyav2NaRA6TGdR?usp=sharing" target="_blank"> AnDI test dataset</a>.
Upload the results using the button below. 
Uploaded files must follow the same syntax as for the AnDi Challenge, as described 
<a href="https://competitions.codalab.org/competitions/23601#learn_the_details-instructions" target="_blank"> here</a>. 
Files must be named 'task1.txt', 'task2.txt', or 'task3.txt', according to the task you are participating in. 
Each task requires a separate upload. 
<br>
<br>
The tool will output a summary of your upload, printing the dimensions for which you successfully participated. 
The results of your submissions will be shown in the tab of the corresponding task, tagged as 'upload'.</p>

""", width = 500, margin = (0,0,20,0))

div_upload_aux = Div(text= "" , width = 100)

# div_mail = Div(text= f""" <h2>Send us a mail!</h2>

# low the same sintaxis as for the AnDi Challenge. You can check the details <a href='https://competitions.codalab.org/competitions/23601#learn_the_details-instructions'>here</a>. 
# Remember, the files' name should be 'task1.tasdas.""")

lnew = grid([ 
            div_upload,
            file_input,
            div_upload_aux])

if notebook:
    def modify_doc(doc):
        doc.add_root(row(lnew, width=10))

    handler = FunctionHandler(modify_doc)
    app = Application(handler)
    show(app)

# T1 vs T2 plot

In [ ]:
 
#| hide

def weighted(cols, weights): 
    return np.average(cols, weights=weights, axis = 0)

def update_main():
    
    # Updating data
    df_dim_t1 = df_t1_results[dims.index(dim_select_main.value)]
    df_dim_t2 = df_t2_results[dims.index(dim_select_main.value)]
    
    df_dim_t1 = df_dim_t1[(df_dim_t1.alpha >= round(alpha_slider_main.value[0],2)) &
                            (df_dim_t1.alpha <= alpha_slider_main.value[1]) &
                            (df_dim_t1.length >= length_slider_main.value[0]) &
                            (df_dim_t1.length <= length_slider_main.value[1]) ]

    df_dim_t2 = df_dim_t2[(df_dim_t2.alpha >= round(alpha_slider_main.value[0], 2)) &
                            (df_dim_t2.alpha <= alpha_slider_main.value[1]) &
                            (df_dim_t2.length >= length_slider_main.value[0]) &
                            (df_dim_t2.length <= length_slider_main.value[1]) ]
    
    

    if model_group_main.value != 'All':
        df_dim_t1 = df_dim_t1[df_dim_t1.model == model_group_main.value]
    if noise_group_main.value != 'All':
        df_dim_t1 = df_dim_t1[df_dim_t1.loc_error == float(noise_group_main.value)]
        df_dim_t2 = df_dim_t2[df_dim_t2.loc_error == float(noise_group_main.value)]
    
    # Weighted average
    weig_p1 = partial(weighted, weights = df_dim_t1['num_traj'])
    update_t1 = df_dim_t1[df_dim_t1.columns[6:]].apply(weig_p1)
    
    weig_p2 = partial(weighted, weights = df_dim_t2['num_traj'])
    update_t2 = df_dim_t2[df_dim_t2.columns[6:]].apply(weig_p2)

    team_list = set(update_t1.keys().tolist())
    team_list.update(update_t2.keys().tolist())
    team_list = list(team_list)


    mae, f1 = [], []
    mae_tooltip, f1_tooltip = [], []
    for team in team_list:
        try: 
            mae.append(update_t1[team])
            mae_tooltip.append(update_t1[team])
        except: 
            mae.append(mae_not)
            mae_tooltip.append(' -')

        try:
            f1.append(update_t2[team])
            f1_tooltip.append(update_t2[team])
        except: 
            f1.append(f1_not)
            f1_tooltip.append(' -')       
        

    d = pd.DataFrame(data = [[t, m, f, mtt, ftt] for t, m, f, mtt, ftt in zip(team_list, mae, f1, mae_tooltip, f1_tooltip)], 
                     columns = ['team', 'mae', 'f1score', 'mae_tt', 'f1_tt'])
    d['index'] = d.index     
    
    # If there are no enough trajectories
    if df_dim_t1['num_traj'].sum() == 0 or df_dim_t2['num_traj'].sum() == 0:
        div_main_aux.text = f"""
        <p style="color:red">  <b>Warning!</b> <br> Not enough trajectories. Select wider option ranges.</p>"""
        return
    
    d.sort_values(by=['mae'], inplace=True)
    d_t1 = d[d['mae'] != mae_not]

    d.sort_values(by=['f1score'], inplace=True)
    d_t2 = d[d['f1score'] != f1_not]
    
    
    source  = ColumnDataSource(d) 
    source_t1  = ColumnDataSource(d_t1)
    source_t2  = ColumnDataSource(d_t2)
    
    # Figure
    mapper = LinearColorMapper(palette = "Turbo256", low = 0, high = len(team_list))   
    fig_main.renderers = []
    fig_main.circle(x="mae", y="f1score", source=source, size=10, 
                    line_color='black', fill_color = {"field": 'index', "transform": mapper}, fill_alpha = 0.6)
    fig_main.tools[-1] = HoverTool(tooltips=TOOLTIPS)
    
    fig_mae.renderers = []    
    fig_mae.x_range.factors = []
    fig_mae.x_range.factors = d_t1['team'].tolist()
    fig_mae.circle(x='team', y = 'mae', size = 10, source = source_t1,  
                        line_color='black', fill_color = {"field": 'index', "transform": mapper}, fill_alpha = 0.6)
    
    fig_f1.renderers = []
    fig_f1.y_range.factors = []
    fig_f1.y_range.factors = d_t2['team'].tolist()    
    fig_f1.circle(y='team', x = 'f1score', size = 10, source = source_t2,  
                        line_color='black', fill_color = {"field": 'index', "transform": mapper}, fill_alpha = 0.6)
    
    
    # Text
    div_main_aux.text = f"""
    <b>Number of trajectories:</b>
    <br>
    Task 1 = {df_dim_t1['num_traj'].sum()}
    <br>
    Task 2 = {df_dim_t2['num_traj'].sum()}"""
    


In [ ]:
 
#| hide

# If there is a change in dimensions, first change the noise options then 
# go for usual update
def update_dim():
    noise_group_main.options = ['All']+[str(n) for n in np.unique(df_t1_results[dims.index(dim_select_main.value)].loc_error)[::-1]]
    noise_group_main.value = 'All'
    
    update_main() 

### Figure

In [ ]:
#| hide
#| exec: false
lengths = np.arange(10,1000,1)
alphas = np.arange(0.05, 2.1, 0.2)
noise = [0.1, 0.5, 1]

mae_not = 0.6; f1_not = 0.3

dim_select_main = Select(title="Dimensions", value="1D", options=dims)

alpha_slider_main = RangeSlider(start=alphas[0], end=2, value=(alphas[0], 2), step=alphas[1]-alphas[0],  title=r"Anomalous exponent")

length_slider_main = RangeSlider(start=lengths[0], end=1000, value=(lengths[0], 1000), step=lengths[1]-lengths[0], title=r"Length")

model_group_main = Select(options=['All']+[p for p in processes], value="All", title ='Diffusion model')
noise_group_main = Select(options=['All']+[str(n) for n in np.unique(df_t1_results[0].loc_error)[::-1]], value="All", title ='Signal to Noise ratio (SNR)')


# Figure
TOOLTIPS=[
    ("Team", "@team"),
    ("MAE", "@mae_tt"),
    ("F1 score", "@f1_tt")
]

fig_main = figure(x_range = [0.025,0.62], y_range = [0.25,1.025], tools= "pan,wheel_zoom,reset,box_zoom", 
                  x_axis_label = 'MAE', y_axis_label = 'F1 score', margin = (0,0,0,20))
fig_main.background_fill_color=None

hover = HoverTool(tooltips=TOOLTIPS)
fig_main.add_tools(hover)

glyph_t1 = Label(x=0.42, y=0.3, text="Task 1 not submitted", text_color = '#949494')
glyph_t2 = Label(x=0.6, y=0.85, text="Task 2 not submitted", text_color = '#949494', angle = 90, angle_units = 'deg')
hline = Span(location=mae_not, dimension='height', line_color='#C5C5C5', line_dash='dashed', line_width=3, level = 'underlay')
vline = Span(location=f1_not, dimension='width', line_color='#C5C5C5', line_dash='dashed', line_width=3, level = 'underlay')
for x in [glyph_t1, glyph_t2, hline, vline]:
    fig_main.add_layout(x)
    
# Horizontal and vertical figures
fig_mae = figure(x_range=[''], plot_width = fig_main.plot_width, plot_height = 200, y_axis_label = 'MAE', y_range = [-0.1,0.6], margin = (0,0,0,20), tools = '')
fig_mae.yaxis.ticker = np.arange(0,0.61,0.1)
fig_mae.xaxis.major_label_orientation = np.pi/4

fig_f1 = figure(y_range=[''], plot_height = fig_main.plot_height, plot_width = 250, x_axis_label = 'F1 score', x_range = [0.15, 1.2], y_axis_location="right", tools = '')
fig_f1.xaxis.ticker = [0.25, 0.5, 0.75, 1]
 

# Heading
# <a href="https://doi.org/10.1117/12.2567914"><img src="http://andi-challenge.org/wp-content/uploads/2021/02/banner_challenge_noborder.png" width="1000", height ="178.328"  ></a>
div = Div(text= """
<br>
<h1>AnDi Challenge interactive tool</h1>

Explore the results of the AnDi challenge interactively. Here you will find access to the details of each submissions results over the different features of the dataset. 
This tab showcases a summary of the results of Task 1 and Task 2. The rest of the tabs corresponds to each of the tasks separatedely.""", 
          width = 1000, margin = (0,0,20,0))


div_main_aux = Div(text= "" , width = 220)




# Updates
update_main() 
controls = [dim_select_main, model_group_main, noise_group_main]
for control in controls:
    if control == dim_select_main:
        control.on_change('value', lambda attr, old, new: update_dim())
    else:
        control.on_change('value', lambda attr, old, new: update_main())
sliders = [alpha_slider_main, length_slider_main]
for control in sliders:
    control.on_change('value_throttled', lambda attr, old, new: update_main())
all_controls = controls+sliders

# Layout
inputs = column(*all_controls+[div_main_aux], width = 200, height = 500)
layout_main = row(inputs, gridplot([[fig_main, fig_f1], [fig_mae, None]], merge_tools=False))
layout_main = column(div, layout_main)

if notebook:
    def modify_doc(doc):
        doc.add_root(row(layout_main, width=3))

    handler = FunctionHandler(modify_doc)
    app = Application(handler)
    show(app)

# Task 3

### Functions

In [ ]:
 
#| hide

def updated_preds_t3(df, team, 
                     model1 = None, model2 = None,
                     a1_min = 0, a1_max = 2, 
                     a2_min = 0, a2_max = 2,
                     cp_min = 1, cp_max = 199):
    # Exponent 1
    df = df.loc[lambda df: (df['alpha1'] >= a1_min) & (df['alpha1'] <= a1_max)]
    # Exponent 2
    df = df.loc[lambda df: (df['alpha2'] >= a2_min) & (df['alpha2'] <= a2_max)]
    # Model 1
    if model1 != 'All' and model1 != None:
        model = processes.index(model1)
        df = df.loc[lambda df: (df['model1'] == model)]
    if model2 != 'All' and model2 != None:
        model = processes.index(model2)
        df = df.loc[lambda df: (df['model2'] == model)]
    # Change point
    df = df.loc[lambda df: (df['change_point'] >= cp_min) & (df['change_point'] <= cp_max)]
    
    filter_col = [col for col in df if col.startswith(team)]
    
    return df[filter_col]

In [ ]:
 
#| hide

def rmse_mod(p):
    
    kwargs = {'df': df_t3[dims.index(dim_select_t3.value)], 
              'a1_min': alpha1_slider_t3.value[0], 'a1_max': alpha1_slider_t3.value[1],
              'a2_min': alpha2_slider_t3.value[0], 'a2_max': alpha2_slider_t3.value[1],
              'cp_min': cp_slider_t3.value[0], 'cp_max': cp_slider_t3.value[1],
              'team': team_select_t3.value}



    model1, model2, rmse, color, posx, posy, text = [], [], [], [], [], [], []

    for idx1, m1 in enumerate(processes):
        for idx2, m2 in enumerate(processes):
            model1.append(m1)
            model2.append(m2)

            df = updated_preds_t3(model1 = m1, model2 = m2, **kwargs)

            preds = df[df.columns[0]]
            trues = df_t3[dims.index(dim_select_t3.value)]['change_point'][preds.index].values
            
            if len(trues) > 0:
                rmse.append(np.sqrt(((preds.values-trues)**2).mean()))          
                text.append(str(round(rmse[-1],2)))
            else:
                rmse.append(0)          
                text.append('  --')
                
            color.append('#FF5733')
            posx.append(idx1+0.3)
            posy.append(4-idx2+0.4)

    rmse = np.array(rmse)
    alpha = (rmse-np.min(rmse[rmse > 0]))/(np.max(rmse)-np.min(rmse[rmse > 0]))+0.1

    source = ColumnDataSource(
        data=dict(model1 = model1, model2 = model2, colors = color, alphas = alpha, posx = posx, posy = posy, text = text)
    )

    p.renderers = []
    rectwidth = 0.9
    p.rect('model1', 'model2', rectwidth, rectwidth, source=source,
          color='colors', alpha='alphas',line_width=1)
    glyph = Text(x="posx", y="posy", text="text", text_color = '#000000')
    p.add_glyph(source, glyph)

In [ ]:
 
#| hide

def rmse_alpha(p):
    
    kwargs = {'df': df_t3[dims.index(dim_select_t3.value)], 
              'model1': model1_group_t3.value, 'model2': model2_group_t3.value,
              'cp_min': cp_slider_t3.value[0], 'cp_max': cp_slider_t3.value[1],
              'team': team_select_t3.value}


    alpha1, alpha2, rmse, color, posx, posy, text = [], [], [], [], [], [], []


    for idx1, a_1 in enumerate(alphas_range[:-1]):
        for idx2, a_2 in enumerate(alphas_range[:-1]):
            alpha1.append(idx1+0.5)
            alpha2.append(idx2+0.5)           

            df = updated_preds_t3(a1_min = a_1, a1_max = a_1+0.5,
                                  a2_min = a_2, a2_max = a_2+0.5,
                                  **kwargs)

            preds = df[df.columns[0]]
            trues = df_t3[dims.index(dim_select_t3.value)]['change_point'][preds.index].values

            if len(trues) > 0:
                rmse.append(np.sqrt(((preds.values-trues)**2).mean()))          
                text.append(str(round(rmse[-1],2)))
            else:
                rmse.append(0)          
                text.append('--')

            color.append('#FF5733')
            posx.append(idx1+0.35)
            posy.append(idx2+0.4)

    rmse = np.array(rmse)

    alpha = (rmse-np.min(rmse[rmse > 0]))/(np.max(rmse)-np.min(rmse[rmse > 0]))+0.1

    source = ColumnDataSource(
        data=dict(alpha1 = alpha1, alpha2 = alpha2, colors = color, alphas = alpha, posx = posx, posy = posy, text = text)
    )
    
    p.renderers = []
    rectwidth = 0.9
    p.rect('alpha1', 'alpha2', rectwidth, rectwidth, source=source,
          color='colors', alpha='alphas',line_width=1)
    glyph = Text(x="posx", y="posy", text="text", text_color = '#000000')
    p.add_glyph(source, glyph)


In [ ]:
 
#| hide

def cp_figures(rmse, mae, f1):
    
    kwargs = {'df': df_t3[dims.index(dim_select_t3.value)], 
                  'model1': model1_group_t3.value, 'model2': model2_group_t3.value,
                  'a1_min': alpha1_slider_t3.value[0], 'a1_max': alpha1_slider_t3.value[1],
                  'a2_min': alpha2_slider_t3.value[0], 'a2_max': alpha2_slider_t3.value[1],
                  'team': team_select_t3.value}

    
    rmse_cp, f1_s1, f1_s2, mae_s1, mae_s2 = [], [], [], [], []
    for idx, cp in enumerate(cut_avg[:-1]):


            df = updated_preds_t3(cp_min = cp, cp_max = cp+20,
                                  **kwargs)
            
            # If there are no enough trajectories
            if df.shape[0] == 0:
                div_t3_aux.text = f"""
                <p style="color:red">  <b>Warning!</b> <br> Not enough trajectories for some analysis. Select wider option ranges.</p>"""
                return
            
            # rmse
            preds = df[df.columns[0]]
            trues = df_t3[dims.index(dim_select_t3.value)]['change_point'][preds.index].values
            rmse_cp.append(np.sqrt(((preds.values-trues)**2).mean()))

            # f1 s1
            preds = df[df.columns[1]]
            trues = df_t3[dims.index(dim_select_t3.value)]['model1'][preds.index].values
            f1_s1.append(f1_score(trues, preds, average = 'micro'))

            # f1 s2
            preds = df[df.columns[3]]
            trues = df_t3[dims.index(dim_select_t3.value)]['model2'][preds.index].values
            f1_s2.append(f1_score(trues, preds, average = 'micro'))

            # mae s1
            preds = df[df.columns[2]]
            trues = df_t3[dims.index(dim_select_t3.value)]['alpha1'][preds.index].values
            mae_s1.append((preds-trues).abs().mean())

            # mae s1
            preds = df[df.columns[4]]
            trues = df_t3[dims.index(dim_select_t3.value)]['alpha2'][preds.index].values
            mae_s2.append((preds-trues).abs().mean())

    rmse.renderers = []
    rmse.line(xticks, rmse_cp, line_width = 2, line_color = '#FF5733')
    rmse.circle(xticks, rmse_cp, line_color = '#FF5733', fill_color="white", size=8)

    f1.renderers = []
    l1 = f1.line(xticks, f1_s1, line_width = 2, line_color = '#36B33F')
    l2 = f1.line(xticks, f1_s2, line_width = 2, line_color = '#1294B3')
    f1.circle(xticks, f1_s1, line_width = 2, line_color = '#36B33F', fill_color="white", size=8)
    f1.circle(xticks, f1_s2, line_width = 2, line_color = '#1294B3', fill_color="white", size=8)
    
    mae.renderers = []
    mae.line(xticks, mae_s1, line_width = 2, line_color = '#36B33F')
    mae.line(xticks, mae_s2, line_width = 2, line_color = '#1294B3')
    mae.circle(xticks, mae_s1, line_width = 2, line_color = '#36B33F', fill_color="white", size=8)
    mae.circle(xticks, mae_s2, line_width = 2, line_color = '#1294B3', fill_color="white", size=8)
    
    
    df = updated_preds_t3(**kwargs)
    div_t3_aux.text = f""" <b>Prediction summary </b>
    <br>
    Number trajs. = {round(df.shape[0],3)}
    <br>
    RMSE = {round(np.mean(rmse_cp),3)} 
    <br>
    MAE = {round(np.mean(mae_s1+mae_s2),3)}
    <br>
    F1-score = {round(np.mean(f1_s1+f1_s2),3)}"""

    

In [ ]:
 
#| hide

def update_t3():
        rmse_mod(fig_rmse_mod_t3)
        rmse_alpha(fig_rmse_alpha_t3)
        cp_figures(fig_rmse_cp_t3, fig_mae_cp_t3, fig_f1_cp_t3)
        
        
def update_t3_mod():   
        rmse_alpha(fig_rmse_alpha_t3)
        cp_figures(fig_rmse_cp_t3, fig_mae_cp_t3, fig_f1_cp_t3)
        
def update_t3_alpha():     
        rmse_mod(fig_rmse_mod_t3)
        cp_figures(fig_rmse_cp_t3, fig_mae_cp_t3, fig_f1_cp_t3)

### Figure

In [ ]:
#| hide
#| exec: false
# Widgets
dim_select_t3 = Select(value="1D", options=dims, title = 'Dimension')
team_select_t3 = Select(value=np.unique([n[:-3] for n in df_t3[0].keys()[6:]])[0], options = np.unique([n[:-3] for n in df_t3[0].keys()[6:]]).tolist(), title = 'Team')

alpha1_slider_t3 = RangeSlider(start=0, end=2, value=(0, 2), step=.05, title=r"Exponent segment 1")
alpha2_slider_t3 = RangeSlider(start=0, end=2, value=(0, 2), step=.05, title=r"Exponent segment 2")


model1_group_t3 = Select(options=['All']+[p for p in processes], value="All", title ='Model segment 1')
model2_group_t3 = Select(options=['All']+[p for p in processes], value="All", title ='Model segment 2')
cp_slider_t3 = RangeSlider(start=1, end=199, value=(0, 199), step=1, title=r"Changepoint")



# Text    
div_t3 = Div(text= f""" <h2>Task 3 - Segmentation of trajectories</h2>
The goal of this task is to find the changepoint in which a trajectory changes either its anomalous diffusion exponent or its diffusion model. 
Participants have to predict the changepoint position and the anomalous diffusion exponent and diffusion model of the two resulting segments.
The upper-central figure shows the RMSE in the changepoint w.r.t. to the diffusion models of the first and second segment.
The lower-central figure shows the same RMSE but w.r.t. to the anomalous diffusion exponent of the first and second segment.
The three rightmost figures show the RMSE, the MAE on the exponent prediction and the F1-score of the model classification, all as a function of the changepoint.
""", width = 1000, margin = (0,0,20,0))

div_t3_aux = Div(text= "" , width = 220)

# Plots
mod_alpha_width = 500
fig_rmse_mod_t3 = figure(tools="",  
                         y_range=sorted(processes, reverse=True), 
                         x_range=processes, x_axis_label = 'Model first segment', y_axis_label = 'Model second segment',
                         plot_width = mod_alpha_width,
                        plot_height = mod_alpha_width, 
                        margin = (0,20,0,20))
fig_rmse_mod_t3.background_fill_color=None
rmse_mod(fig_rmse_mod_t3)


    
alphas_range = [0.05,0.5,1,1.5,2]   
fig_rmse_alpha_t3 = figure(tools="",
                      y_range = [f'{round(alphas_range[idx],2)}-{round(alphas_range[idx+1],2)}' for idx, _ in enumerate(alphas_range[:-1])],
                      x_range = [f'{round(alphas_range[idx],2)}-{round(alphas_range[idx+1],2)}' for idx, _ in enumerate(alphas_range[:-1])],
                      x_axis_label = 'Exponent first segment', y_axis_label = 'Exponent second segment',
                      plot_width = mod_alpha_width,
                      plot_height = mod_alpha_width,
                      margin = (0,20,0,20))
fig_rmse_alpha_t3.background_fill_color=None
rmse_alpha(fig_rmse_alpha_t3)


cut_avg = np.arange(10, 201, 20)
cp_width = 400
xticks = [f'{i}-{i+20}' for i in cut_avg[:-1]]
fig_rmse_cp_t3 = figure(tools="",
                        plot_width=cp_width, plot_height = int(0.8*cp_width), 
                        x_range = xticks, x_axis_label = 'Changepoint',
                        y_axis_label = 'RMSE changepoint',
                        margin = (20,0,0,0))
fig_f1_cp_t3 = figure(tools="",
                      plot_width=cp_width, plot_height = int(0.8*cp_width), 
                      x_range = xticks, x_axis_label = 'Changepoint', 
                      y_axis_label = 'F1-score diffusion model',
                      margin = (20,0,0,0))
fig_mae_cp_t3 = figure(tools="",
                       plot_width=cp_width, plot_height = int(0.8*cp_width),
                       x_range = xticks, x_axis_label = 'Changepoint',
                       y_axis_label = 'MAE anomalous exponent',
                       margin = (20,0,0,0))
fig_rmse_cp_t3.background_fill_color=None
fig_f1_cp_t3.background_fill_color=None
fig_mae_cp_t3.background_fill_color=None

glyph_s1 = Label(x=50, y=cp_width/2, x_units = 'screen', y_units = 'screen', text="First segment", text_color = '#36B33F')
glyph_s2 = Label(x=50, y=cp_width/2-20, x_units = 'screen', y_units = 'screen', text="Second segment", text_color = '#1294B3')
for g in [glyph_s1, glyph_s2]:
    fig_mae_cp_t3.add_layout(g)

cp_figures(fig_rmse_cp_t3, fig_mae_cp_t3, fig_f1_cp_t3)

for fig in [fig_rmse_cp_t3, fig_mae_cp_t3, fig_f1_cp_t3]:
    fig.xaxis.major_label_orientation = np.pi/4

# Updates
for control in [dim_select_t3, team_select_t3]:
    control.on_change('value', lambda attr, old, new: update_t3())
    
cp_slider_t3.on_change('value_throttled', lambda attr, old, new: update_t3())

for control in [model1_group_t3, model2_group_t3]:
    control.on_change('value', lambda attr, old, new: update_t3_mod())
    
for control in [alpha1_slider_t3, alpha2_slider_t3]:
    control.on_change('value_throttled', lambda attr, old, new: update_t3_alpha())
    

def update_select_team_t3(attrname, old, new):    
    team_select_t3.options = np.unique([n[:-3] for n in df_t3[dims.index(dim_select_t3.value)].keys()[6:]]).tolist() 
    team_select_t3.value = team_select_t3.options[0]
    update_t3()
    
dim_select_t3.on_change('value', update_select_team_t3)

inputs = column(dim_select_t3,
                team_select_t3,                
                row(model1_group_t3, model2_group_t3, width = 220),
                cp_slider_t3,
                alpha1_slider_t3,
                alpha2_slider_t3,
                div_t3_aux, width = 220)
    
col_rmse = column(fig_rmse_mod_t3, fig_rmse_alpha_t3)
col_cp = column(fig_rmse_cp_t3, fig_mae_cp_t3, fig_f1_cp_t3)

l3 = grid([div_t3, 
           row(inputs, col_rmse, col_cp)])

if notebook:
    def modify_doc(doc):
        doc.add_root(row(l3, width=10))

    handler = FunctionHandler(modify_doc)
    app = Application(handler)
    show(app)

# Distribution and merge

In [ ]:
#| hide
#| exec: false
tab_main = Panel(child = layout_main, title = 'Challenge Summary')
tab1 = Panel(child=l1,title="Task 1")
tab2 = Panel(child=l2,title="Task 2")
tab3 = Panel(child=l3,title="Task 3")
tab4 = Panel(child=lnew,title="New submission")

layout = Tabs(tabs=[tab_main, tab1, tab2, tab3, tab4])

if notebook:
    def modify_doc(doc):
        doc.add_root(row(layout, width=10))

    handler = FunctionHandler(modify_doc)
    app = Application(handler)
    show(app)

# For export
curdoc().add_root(row(layout, width=800))

# To .py

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()